In [1]:
import pandas as pd
metadata=pd.read_csv("~/dance/dance/metadata/scdeepsort.csv",header=0, skiprows=[i for i in range(1, 70)])

In [2]:
atlas_metadata=metadata[metadata.columns[~metadata.columns.isin(['celltype_fname','celltype_url'])]]

In [3]:
atlas_metadata['data_fname'].iloc[0]

'train_human_Blood471647b3-04fe-4c76-8372-3264feb950e8_data.h5ad'

In [ ]:
METADIR ="/home/zyxing/dance/dance/metadata"
import numpy as np
atlas_metadata['dataset_id']=np.nan
tissues=['blood','brain','heart','intestine','kidney','lung','pancreas']
for tissue in tissues:
    def find_dataset_in_metadata(datasets, tissue):
        datasets_in_metadata = []
        for dataset_id in datasets:
            all_datasets = pd.read_csv(f"{METADIR}/scdeepsort.csv", header=0, skiprows=[i for i in range(1, 68)])
            for collect_dataset in all_datasets[all_datasets["tissue"] == tissue]["data_fname"].tolist():
                if dataset_id in collect_dataset:
                    datasets_in_metadata.append(
                        (collect_dataset.split(tissue)[1] +
                            (tissue + collect_dataset.split(tissue)[2] if len(collect_dataset.split(tissue)) >= 3 else '')
                            ).split('_')[0])
                    break
        return datasets_in_metadata
    atlas_metadata.loc[atlas_metadata['tissue']==tissue.capitalize(),'dataset_id']=find_dataset_in_metadata(atlas_metadata[atlas_metadata['tissue']==tissue.capitalize()]['data_fname'],tissue.capitalize())
atlas_metadata.loc[:,'dataset_id']=atlas_metadata['dataset_id'].str.split('(').str.get(0)
atlas_metadata.loc[:,'dataset_tissue']=atlas_metadata['dataset_id']+"_"+atlas_metadata['tissue']

/tmp/ipykernel_2362057/2040604470.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atlas_metadata['dataset_id']=np.nan
/tmp/ipykernel_2362057/2040604470.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['471647b3-04fe-4c76-8372-3264feb950e8', '84230ea4-998d-4aa8-8456-81dd54ce23af', '8a554710-08bc-4005-87cd-da9675bdc2e7', 'd3566d6a-a455-4a15-980f-45eb29114cab', 'eeacb0c1-2217-4cf6-b8ce-1f0fedf1b569', '01209dce-3575-4bed-b1df-129f57fbc031', '055ca631-6ffb-40de-815e-b931e10718c0', '2a498ace-872a-4935-984b-1afa70fd9886', '2adb1f8a-a6b1-4909-8ee8-484814e2d4bf', '3faad104-2ab8-4434-816d-474d8d2641db', '4c4cd77c-8fee-4836-9145-16562a8782fe', 'ae29ebd0-1973-40a4-a6a

In [5]:
atlas_metadata.loc[:,'dataset_tissue']

0         471647b3-04fe-4c76-8372-3264feb950e8_Blood
1         84230ea4-998d-4aa8-8456-81dd54ce23af_Blood
2         8a554710-08bc-4005-87cd-da9675bdc2e7_Blood
3         d3566d6a-a455-4a15-980f-45eb29114cab_Blood
4         eeacb0c1-2217-4cf6-b8ce-1f0fedf1b569_Blood
                           ...                      
132    3294d050-6eeb-4a00-b24c-71aacc9b777f_Pancreas
133    2adb1f8a-a6b1-4909-8ee8-484814e2d4bf_Pancreas
134    ff45e623-7f5f-46e3-b47d-56be0341f66b_Pancreas
135    66d15835-5dc8-4e96-b0eb-f48971cb65e8_Pancreas
136    97a17473-e2b1-4f31-a544-44a60773e2dd_Pancreas
Name: dataset_tissue, Length: 137, dtype: object

In [12]:
SIMILARITYDIR="/home/zyxing/dance/examples/atlas/sc_similarity_examples"
all_datasets = []
for tissue in tissues:
    dataset_info=pd.read_excel(f"{SIMILARITYDIR}/data/Cell Type Annotation Atlas.xlsx",sheet_name=tissue,index_col=0)
    dataset_info['tissue'] = tissue.capitalize()
    all_datasets.append(dataset_info)
combined_dataset = pd.concat(all_datasets, ignore_index=True)
combined_dataset.loc[:,'dataset_tissue']=combined_dataset['dataset_id']+"_"+combined_dataset['tissue']
combined_dataset=combined_dataset[combined_dataset['queryed']==False]

In [13]:
combined_dataset.shape

(93, 45)

In [14]:
dataset_all=combined_dataset.set_index('dataset_tissue').join(atlas_metadata.set_index('dataset_tissue'),how='left',rsuffix="cta")

In [15]:
dataset_all.shape

(93, 51)

In [16]:
dataset_all.to_csv('atlas_metadata.csv')

In [ ]:
# import crud, models, schemas, auth
# from database import SessionLocal, engine
# from fastapi import Depends
# db = SessionLocal()
# try:
#     for i,row in atlas_metadata.iterrows():
#         dataset_data = schemas.DatasetCreate(tissue_info=row['tissue'], description='atlas')
#         crud.create_dataset(
#             db=db, 
#             dataset=dataset_data, 
#             filename=row['data_fname'], # We use the h5ad filename as the primary name
#             h5ad_file_path=row['data_url'],
#             csv_file_path=None,
#             user_id=-1,
#             is_public=False,
#             is_atlas=True
#         )
#         db.commit()
#         print(f"Files for '{row['data_fname']}' uploaded successfully")
# except Exception as e:
#     # 5. 如果在循环中发生任何错误，回滚事务，撤销所有未提交的更改
#     print(f"\nAn error occurred: {e}")
#     print("Rolling back database transaction.")
#     db.rollback()

# finally:
#     # 6. 无论成功还是失败，最后都要关闭会话
#     print("Closing database session.")
#     db.close()

In [1]:

import pandas as pd
from sqlalchemy.orm import Session
import numpy as np  # 导入 numpy 用于检查 NaN
import os
import sys

# 将项目根目录添加到 Python 路径中，以便可以导入 app 模块
# 这假设你的项目结构是 .../your_project/scripts/populate_atlas.py
# 并且你的 FastAPI 应用在 .../your_project/app/ 目录下
# 如果结构不同，请相应调整路径
BACKEND_URL = "static"
UMAP_DIR = "umaps"
PATTERN_URL = "atlas_pattern"
PATTERN_CSV_URL = "atlas_pattern_csv"

import crud, models, schemas
from database import SessionLocal, engine

# 确保所有表都已创建
models.Base.metadata.create_all(bind=engine)

def get_value(row, column_name):
    """
    安全地从 Pandas 行中获取值。
    如果值是 NaN 或列不存在，返回 None。
    """
    if column_name in row and pd.notna(row[column_name]):
        # 将 numpy 类型转换为 Python 原生类型
        value = row[column_name]
        if isinstance(value, np.generic):
            return value.item()
        return value
    return None

def populate_atlas_from_csv(csv_path: str):
    """
    从 CSV 文件读取 Atlas 元数据并填充到数据库中。
    """
    if not os.path.exists(csv_path):
        print(f"Error: CSV file not found at {csv_path}")
        return

    df = pd.read_csv(csv_path)
    db: Session = SessionLocal()
    
    print(f"Found {len(df)} records in {csv_path}. Starting population...")
    
    populated_count = 0
    try:
        print("\n--- Phase 1: Creating missing Dataset records ---")
        new_datasets_map = {} # 用于存储新创建的 dataset，键为 dataset_name
        for index, row in df.iterrows():
            # 检查数据集是否已存在，避免重复填充
            # 我们用 dataset_name 作为唯一标识符
            dataset_name = get_value(row,'dataset_tissue')
            if not dataset_name:
                print(f"Skipping row with no dataset_name: {row.to_dict()}")
                continue
                
            existing_atlas = db.query(models.AtlasMetadata).filter(models.AtlasMetadata.dataset_name == dataset_name).first()
            if existing_atlas:
                print(f"Skipping already populated dataset: {dataset_name}")
                continue

            # 1. 创建一个 "Dataset" 记录
            dataset_schema = schemas.DatasetCreate(
                tissue_info=get_value(row, 'tissue') or "N/A", # 提供一个默认值
                description=get_value(row, 'data_fname') or "Atlas Dataset"
            )
            umap_csv_path = f"{UMAP_DIR}/{get_value(row, 'tissue').lower()}_{get_value(row, 'dataset_id')}.csv"
            db_dataset = crud.create_dataset(
                db=db,
                dataset=dataset_schema,
                filename= f"{dataset_name}.h5ad",
                dataset_name=dataset_name,
                h5ad_file_path=f"{get_value(row, 'data_url')}", # 假设 Atlas 文件有特殊路径
                user_id=-1,
                is_atlas=True,
                is_public=False,
                umap_csv_path=umap_csv_path
            )
            
            # 2. 将其标记为 Atlas
            db_dataset.is_atlas = True
            new_datasets_map[dataset_name] = db_dataset
         # 提交所有新的 Dataset 记录
        if new_datasets_map:
            print(f"\nCommitting {len(new_datasets_map)} new Dataset records...")
            db.commit()
            print("Phase 1 commit successful.")
            # 提交后，所有 new_datasets_map 中的对象都会自动获得 ID
        else:
            print("No new Datasets to create in Phase 1.")
        print("\n--- Phase 2: Creating missing AtlasMetadata records ---")
        
        for _, row in df.iterrows():
            dataset_name = get_value(row, 'dataset_tissue')
            if not dataset_name:
                continue

            # 再次检查 AtlasMetadata 是否存在，以防万一
            existing_atlas = db.query(models.AtlasMetadata).filter(models.AtlasMetadata.dataset_name == dataset_name).first()
            if existing_atlas:
                continue
             # 从数据库中查找对应的 Dataset (它现在肯定存在了)
            # 这一步至关重要，确保我们获取的是最新的、带有 ID 的对象
            target_dataset = db.query(models.Dataset).join(models.AtlasMetadata, isouter=True).filter(
                models.Dataset.is_atlas == True,
                models.AtlasMetadata.id == None, # 确保它还没有 atlas_metadata
                # 这里我们假设 filename 是唯一的，或者用更可靠的标识符
                models.Dataset.filename == f"{dataset_name}.h5ad"
            ).first()

            if not target_dataset:
                print(f"Error: Could not find a matching dataset for '{dataset_name}' in Phase 2. This should not happen.")
                continue
            tissue = get_value(row, 'tissue')
            number_of_cells = get_value(row, 'Number of Cells')
            dataset_id =get_value(row, 'dataset_id')
            
            
            preview_url = None
            if tissue and number_of_cells and dataset_id:
                preview_url = f"{BACKEND_URL}/{str(tissue).lower()}/{'small' if int(number_of_cells)<10000 else 'large' }/{dataset_id}.png"
            
            positive_pattern_url = None
            negative_pattern_url = None
            positive_pattern_csv_url = None
            negative_pattern_csv_url = None
            if tissue and dataset_id:
                positive_pattern_url = f"{PATTERN_URL}/{str(tissue).lower()}_{dataset_id}_dataset_pattern_positive.png"
                negative_pattern_url = f"{PATTERN_URL}/{str(tissue).lower()}_{dataset_id}_dataset_pattern_negative.png"
                positive_pattern_csv_url = f"{PATTERN_CSV_URL}/{str(tissue).lower()}_{dataset_id}_dataset_pattern_positive.csv"
                negative_pattern_csv_url = f"{PATTERN_CSV_URL}/{str(tissue).lower()}_{dataset_id}_dataset_pattern_negative.csv"
            print(f"{positive_pattern_url}")
            
            
            
            print(f"Preparing AtlasMetadata for Dataset ID: {target_dataset.id} ('{dataset_name}')")
            atlas_meta = models.AtlasMetadata(
                dataset_id=target_dataset.id,
                 # --- 填充新字段 ---
                preview_image_url=preview_url,
                postive_pattern_image_url=positive_pattern_url,
                negative_pattern_image_url=negative_pattern_url,
                postive_pattern_csv_url=positive_pattern_csv_url,
                negative_pattern_csv_url=negative_pattern_csv_url,
                # --- 基础元数据 ---
                species=get_value(row, 'species'),
                tissue=get_value(row, 'tissue'),
                dataset_col=get_value(row, 'dataset'),
                split=get_value(row, 'split'),
                data_fname=get_value(row, 'data_fname'),
                data_url=get_value(row, 'data_url'),
                dataset_name=dataset_name,
                dataset_id_col=get_value(row, 'dataset_id'),

                # --- 数值统计数据 ---
                unnamed_0=get_value(row, 'Unnamed: 0'),
                number_of_cells=get_value(row, 'Number of Cells'),
                number_of_genes=get_value(row, 'Number of Genes'),
                number_of_cell_types=get_value(row, 'Number of Cell Types'),
                number_of_non_zero_entries=get_value(row, 'Number of Non-Zero Entries in X'),
                cell=get_value(row, 'cell'),
                sampled_cell=get_value(row, 'sampled_cell'),

                # --- 细胞和疾病相关信息 ---
                cell_type=get_value(row, 'cell_type'),
                disease_origin=get_value(row, 'disease_origin'),
                tissuecta=get_value(row, 'tissuecta'),
                cell_type_sampled=get_value(row, 'cell_type (sampled)'),
                disease_sampled=get_value(row, 'disease_sampled'),
                assay=get_value(row, 'assay'),
                assay_sampled=get_value(row, 'assay_sampled'),
                dataset_idcta=get_value(row, 'dataset_idcta'),
                queryed=get_value(row, 'queryed'),

                # --- CTA Actinn 相关字段 ---
                cta_actinn=get_value(row, 'cta_actinn'),
                cta_actinn_best_yaml=get_value(row, 'cta_actinn_best_yaml'),
                cta_actinn_best_res=get_value(row, 'cta_actinn_best_res'),
                cta_actinn_run_stats=get_value(row, 'cta_actinn_run_stats'),
                cta_actinn_check=get_value(row, 'cta_actinn_check'),
                cta_actinn_step2_best_yaml=get_value(row, 'cta_actinn_step2_best_yaml'),
                cta_actinn_step2_best_res=get_value(row, 'cta_actinn_step2_best_res'),
                
                # --- CTA Celltypist 相关字段 ---
                cta_celltypist=get_value(row, 'cta_celltypist'),
                cta_celltypist_best_yaml=get_value(row, 'cta_celltypist_best_yaml'),
                cta_celltypist_best_res=get_value(row, 'cta_celltypist_best_res'),
                cta_celltypist_run_stats=get_value(row, 'cta_celltypist_run_stats'),
                cta_celltypist_check=get_value(row, 'cta_celltypist_check'),
                cta_celltypist_step2_best_yaml=get_value(row, 'cta_celltypist_step2_best_yaml'),
                cta_celltypist_step2_best_res=get_value(row, 'cta_celltypist_step2_best_res'),

                # --- CTA scDeepsort 相关字段 ---
                cta_scdeepsort=get_value(row, 'cta_scdeepsort'),
                cta_scdeepsort_best_yaml=get_value(row, 'cta_scdeepsort_best_yaml'),
                cta_scdeepsort_best_res=get_value(row, 'cta_scdeepsort_best_res'),
                cta_scdeepsort_run_stats=get_value(row, 'cta_scdeepsort_run_stats'),
                cta_scdeepsort_check=get_value(row, 'cta_scdeepsort_check'),
                cta_scdeepsort_step2_best_yaml=get_value(row, 'cta_scdeepsort_step2_best_yaml'),
                cta_scdeepsort_step2_best_res=get_value(row, 'cta_scdeepsort_step2_best_res'),

                # --- CTA SingleCellNet 相关字段 ---
                cta_singlecellnet=get_value(row, 'cta_singlecellnet'),
                cta_singlecellnet_best_yaml=get_value(row, 'cta_singlecellnet_best_yaml'),
                cta_singlecellnet_best_res=get_value(row, 'cta_singlecellnet_best_res'),
                cta_singlecellnet_run_stats=get_value(row, 'cta_singlecellnet_run_stats'),
                cta_singlecellnet_check=get_value(row, 'cta_singlecellnet_check'),
                cta_singlecellnet_step2_best_yaml=get_value(row, 'cta_singlecellnet_step2_best_yaml'),
                cta_singlecellnet_step2_best_res=get_value(row, 'cta_singlecellnet_step2_best_res'),
            )
            db.add(atlas_meta)
            populated_count += 1
        
        # 提交所有更改
        db.commit()
        print(f"Successfully populated {populated_count} new atlas datasets.")
        
    except Exception as e:
        print(f"An error occurred: {e}")
        db.rollback() # 如果出错，回滚所有操作
    finally:
        db.close()

if __name__ == "__main__":
    # 将此路径替换为你的实际 CSV 文件路径
    # 脚本会从项目根目录寻找这个文件
    csv_file_path = "atlas_metadata.csv"
    
    # 确保你有一个 admin 用户，其 id 为 1
    # 如果没有，你需要先创建一个
    # (这里省略了创建用户的代码，假设已存在)
    
    populate_atlas_from_csv(csv_file_path)

/home/zyxing/anaconda3/envs/dance-app-backend/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


Found 93 records in atlas_metadata.csv. Starting population...

--- Phase 1: Creating missing Dataset records ---

Committing 93 new Dataset records...
Phase 1 commit successful.

--- Phase 2: Creating missing AtlasMetadata records ---
atlas_pattern/blood_3faad104-2ab8-4434-816d-474d8d2641db_dataset_pattern_positive.png
Preparing AtlasMetadata for Dataset ID: 1 ('3faad104-2ab8-4434-816d-474d8d2641db_Blood')
atlas_pattern/blood_2adb1f8a-a6b1-4909-8ee8-484814e2d4bf_dataset_pattern_positive.png
Preparing AtlasMetadata for Dataset ID: 2 ('2adb1f8a-a6b1-4909-8ee8-484814e2d4bf_Blood')
atlas_pattern/blood_d9b4bc69-ed90-4f5f-99b2-61b0681ba436_dataset_pattern_positive.png
Preparing AtlasMetadata for Dataset ID: 3 ('d9b4bc69-ed90-4f5f-99b2-61b0681ba436_Blood')
atlas_pattern/blood_055ca631-6ffb-40de-815e-b931e10718c0_dataset_pattern_positive.png
Preparing AtlasMetadata for Dataset ID: 4 ('055ca631-6ffb-40de-815e-b931e10718c0_Blood')
atlas_pattern/blood_bc2a7b3d-f04e-477e-96c9-9d5367d5425c_dataset